In [0]:
%pip install loguru --quiet

In [0]:
import json
import random
import time
from datetime import datetime, timedelta
from typing import Final
from loguru import logger

# --- Configuration ---
##### IMPORTANT: Before the demo, replace this with your actual Volume path
##### e.g., `/Volumes/catalog/schema/databricks-demo/iot_landing/`

In [0]:
dbutils.widgets.text(
    "landing_dir",
    "/Volumes/demos/smart_factory/iot_landing",
    "Volume where the IoT landing data will be written to"
)

In [0]:
LANDING_ZONE_PATH: Final[str] = dbutils.widgets.get("landing_dir")
logger.info(f"Landing directory set to: '{LANDING_ZONE_PATH}'")
dbutils.fs.mkdirs(LANDING_ZONE_PATH)

# --- Data Generation Logic ---

In [0]:
device_ids = [f"MFG-A-{1000+i}" for i in range(10)] + [f"MFG-B-{2000+i}" for i in range(5)]

In [0]:
def generate_sensor_reading(device_id):
    """Generates a single sensor reading."""
    # 95% of data is normal
    if random.random() < 0.95:
        temp = round(random.uniform(25.0, 35.0), 2)
        vibration = round(random.uniform(0.1, 0.5), 4)
    # 5% of data is anomalous (potential issue)
    else:
        temp = round(random.uniform(40.0, 45.0), 2)
        vibration = round(random.uniform(0.5, 1.2), 4)

    return {
        "device_id": device_id,
        "timestamp": (datetime.utcnow() - timedelta(seconds=random.randint(0, 60))).isoformat(),
        "temperature_celsius": temp,
        "vibration_hz": vibration,
        "location": "Factory Floor A" if "MFG-A" in device_id else "Factory Floor B"
    }

In [0]:
def generate_and_save_batch(batch_size=100):
    """Generates a batch of data and saves it as a single JSON file."""
    data = [generate_sensor_reading(random.choice(device_ids)) for _ in range(batch_size)]
    file_name = f"sensor-batch-{int(time.time())}.json"
    file_path = f"{LANDING_ZONE_PATH.rstrip('/')}/{file_name}"

    with open(file_path, "w") as f:
        json.dump(data, f)
    logger.info(f"Generated batch of {len(data)} records to: {LANDING_ZONE_PATH}{file_name}")

# --- Generate Initial Data ---

In [0]:
logger.info(f"Generating data in: {LANDING_ZONE_PATH}")
logger.info("-" * 30)
for i in range(5):
    generate_and_save_batch()
    time.sleep(1)

In [0]:
logger.info("-" * 30)
logger.info("Listing files in landing zone:")
display(dbutils.fs.ls(LANDING_ZONE_PATH))